In [2]:
import textgrid

praat_file_path = 'data/transcriptions/audio1913346482_Fadhl_revised.TextGrid'

grid = textgrid.TextGrid.fromFile(praat_file_path)

In [3]:
import pandas as pd

def parse_tier(tier):
    return [grid.getFirst(tier).__dict__['intervals'][x].__dict__ for x,_interval in enumerate(grid.getFirst(tier))]


parsed_tiers = []
for tier in grid.getNames():
    if 'Comments' in tier:
        pass
    else:
        df = pd.DataFrame.from_dict(parse_tier(tier))
        df.index = df.apply(lambda x: pd.Interval(x['minTime'], x['maxTime']), axis=1)
        parsed_tiers.append(df)

grid_series = pd.concat(parsed_tiers).sort_index().replace('', pd.NA).dropna()['mark']
grid_series

/var/folders/nb/rn_wr53j4mbcyl_kw5fw8t3m0000gn/T/ipykernel_11369/1492480716.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


(1.12, 5.197]           %um Ok and Amna if you could read the the seco...
(6.106, 28.413]         Ok %aa نود أيضا تشجيع المزيد من الطلاب الإمارا...
(28.878, 62.887]        Ok great, right. So, at %aa so as I said at %a...
(66.092, 69.105]                Yes, it's very similar to the last email.
(69.286, 97.564]        It is yeah but when.. if you think about how %...
                                              ...                        
(3243.227, 3253.588]    Yeah, so yeah because the students nowadays al...
(3253.905, 3260.553]    %um But in the past I think Emirati students a...
(3259.904, 3261.618]                                    Yes, in the past.
(3260.501, 3267.227]    Yeah, a a lot of people. Yeah, yeah a lot of p...
(3267.071, 3275.485]    %um Right. Well, things change. Ok, right. So,...
Name: mark, Length: 441, dtype: object

In [170]:
## crop audio file and create test.wav sample
from scipy.io import wavfile
from IPython.display import Audio


# def crop_audio(signal, rate, start=0, end=None):
#     """Crops audio signal to specified start and end times in minutes"""
#     duration = len(signal)/rate  # in seconds
#     if end is None:
#         end = duration

#     start = start*60
#     end = end*60
#     start_point = start*rate
#     sample_length = end*rate
#     return signal[start_point:sample_length]




rate, signal = wavfile.read('data/sound_files/audio1913346482.wav')

# # end is in minutes
# wavfile.write('data/sound_files/audio1913346482_cropped_last20minutes.wav', rate,
#               crop_audio(signal, rate, start=34, end=54))

# #load test.wav
# rate, signal = wavfile.read(
#     'data/sound_files/audio1913346482_cropped_last20minutes.wav')

In [171]:
import stable_whisper
import torch


device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = stable_whisper.load_model(model_path, device=device)

## loading default tiny model
model = stable_whisper.load_model('tiny', device=device)


In [173]:
# get annotation transcription, and crop to number of minutes

# #last 20 minutes
# crop_start = 34
# crop_end = 54
# cropped_transcript = ' '.join(
#     grid_series.loc[crop_start*60:crop_end*60].to_list())
# aligned_result = model.align(
#     'data/sound_files/audio1913346482_cropped_last20minutes.wav', cropped_transcript, language='ar')

full_transcript = ' '.join(grid_series.loc[:].to_list()) # full transcript

aligned_result = model.align(
    'data/sound_files/audio1913346482.wav', full_transcript, language='ar')

Align: 100%|██████████| 3274.14/3274.14 [00:36<00:00, 89.51sec/s] 
/var/folders/nb/rn_wr53j4mbcyl_kw5fw8t3m0000gn/T/ipykernel_11369/618292503.py:13: UserWarning: Failed to align the last 4/6073 words after 54:34.120.
  aligned_result = model.align(
/var/folders/nb/rn_wr53j4mbcyl_kw5fw8t3m0000gn/T/ipykernel_11369/618292503.py:13: UserWarning: 36/909 segments failed to align.
  aligned_result = model.align(


In [174]:

segment_index = 4 # change to test different segments

words = aligned_result.to_dict()['segments'][segment_index]['words']

wordslice = slice(0, 16)  # pick random slice of words to test

def get_audio_segment(words):
    start = words[0]['start']
    end = words[-1]['end']
    return signal[int(round(start*rate, 0)):int(round(end*rate, 0))].T

print([(x['start'],x['end'],x['word']) for x in words[wordslice]])
Audio(get_audio_segment(words[wordslice]), rate=rate)

[(16.84, 17.2, ' الطلاب'), (17.2, 17.96, ' الإماراتيين'), (18.0, 18.16, ' خاصة'), (18.42, 18.92, ' الطالبات.')]


In [180]:
ma77ad_timestamps = words[words['word'].str.contains('محد')]

ma77ad_timestamps

,word,start,end,probability,tokens,segment_id,id
462,محد,220.02,220.02,0.016706,"[3714, 24401]",71,1
5272,محد,3143.98,3144.46,0.159544,"[3714, 24401]",841,1


In [181]:
def get_audio_slice(start, end, signal, rate, audio_pad_margin=(-1, 1)):
    start = int((start*rate)+(audio_pad_margin[0]*rate))
    end = int((end*rate)+(audio_pad_margin[1]*rate))

    return Audio(signal[start:end], rate=rate)

In [182]:
ma77ad_timestamps_values = ma77ad_timestamps[['start','end']].values

get_audio_slice(*ma77ad_timestamps_values[1], signal, rate, audio_pad_margin=(-0.5, 0.5))

In [216]:
shu_timestamps = words[words['word']=='شو']

shu_timestamps

,word,start,end,probability,tokens,segment_id,id
429,شو,201.78,202.50,0.059086,"[13412, 2407]",67,6
519,شو,263.56,263.76,0.246669,"[13412, 2407]",84,1
627,شو,330.84,331.00,0.266689,"[13412, 2407]",104,4
637,شو,334.04,334.26,0.198830,"[13412, 2407]",104,14
690,شو,364.82,365.90,0.297611,"[13412, 2407]",114,2
710,شو,376.78,376.78,0.342951,"[13412, 2407]",118,1
732,شو,385.92,386.12,0.409265,"[13412, 2407]",120,1
830,شو,440.30,440.60,0.025810,"[13412, 2407]",133,1
891,شو,472.78,473.04,0.020100,"[13412, 2407]",146,1
1019,شو,544.14,544.40,0.167004,"[13412, 2407]",168,2


In [217]:
shu_timestamps_values = shu_timestamps[['start','end']].values

# e.g compare 11 and 13
segment_index = 11
# segment_index = 13
get_audio_slice(*shu_timestamps_values[segment_index], signal, rate, audio_pad_margin=(-0.5, 0.5))

In [218]:
shsmh_timestamps = words[words['word'] == 'شاسمه']

shsmh_timestamps

,word,start,end,probability,tokens,segment_id,id
828,شاسمه,438.04,439.28,0.054770,"[13412, 32277, 2304, 3224]",132,1
987,شاسمه,524.48,525.34,0.070602,"[13412, 32277, 2304, 3224]",163,3
1088,شاسمه,587.84,588.02,0.131819,"[13412, 32277, 2304, 3224]",180,2
1475,شاسمه,869.54,870.70,0.097609,"[13412, 32277, 2304, 3224]",264,0
1969,شاسمه,1221.68,1222.06,0.098255,"[13412, 32277, 2304, 3224]",355,3
2055,شاسمه,1304.40,1305.44,0.051104,"[13412, 32277, 2304, 3224]",366,1
2174,شاسمه,1383.18,1383.72,0.063747,"[13412, 32277, 2304, 3224]",391,2


In [228]:
shsmh_timestamps_values = shsmh_timestamps[['start', 'end']].values

segment_index = 0
get_audio_slice(*shsmh_timestamps_values[segment_index],
                signal, rate, audio_pad_margin=(-0.5, 0.5))

In [232]:
w7da_timestamps = words[words['word'] == 'واحدة']

w7da_timestamps

,word,start,end,probability,tokens,segment_id,id
594,واحدة,312.26,312.52,0.529171,"[36764, 24401, 3660]",99,5
4103,واحدة,2511.28,2511.70,0.586797,"[36764, 24401, 3660]",685,11


In [237]:
w7da_timestamps_values = w7da_timestamps[['start', 'end']].values

segment_index = 0
get_audio_slice(*w7da_timestamps_values[segment_index],
                signal, rate, audio_pad_margin=(-0.5, 0.5))